# Text preprocessing - Pipeline

This code will preprocess a pair of documents in order to analyse their similarity afterwards.

Import libraries

In [1]:
import pandas as pd
import json
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Read files

In [6]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
trainv1 = pd.read_csv('dados/train v0.1.csv')
trainv1.head() 

C:\Users\lucas\AppData\Local\Temp/ipykernel_4412/2981112181.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


### Carregar noticias a partir dos ids:

Functions that read json documents

In [13]:
#Function reads a json file
def readJsonFile(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

    
#Function returns a dataframe with the text of the pairs
def getJsonDocumentPair(dataPath, pairId):
    listIds = pairId.split('_')
    doc1Id = listIds[0]
    doc2Id = listIds[1]

    doc1Path = dataPath + doc1Id[-2:] + '/' + doc1Id + '.json' 
    doc2Path = dataPath + doc2Id[-2:] + '/' + doc2Id + '.json' 

    doc1Json = readJsonFile(doc1Path)
    doc2Json = readJsonFile(doc2Path)

    return (doc1Json, doc2Json)

Getting text to preprocess

In [20]:
dataPath = '../Dados/train v0.1/'
#jsonPair = getJsonDocumentPair(dataPath,'1484084337_1484110209')
#textDoc1 = jsonPair[0]['text']

ids = trainv1['pair_id']
for id in ids:
    jsonPair = getJsonDocumentPair(dataPath, id)
    textDoc1 = jsonPair[0]['text']
    # pegar outras informacoes alem de texto?
    # montar um dataframe com as informaçoes 

#Creating DF to text
dfText = pd.DataFrame([[textDoc1]], columns=['original_text'])
dfText

,original_text
0,"MARTINSBURG, W.Va. — A suspected drunken drive..."


Removing punctuation

In [19]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
#Function that removes punctuation 
def removePunctuation(text):
    punctuationFreeDoc = "".join([i for i in text if i not in string.punctuation])
    return punctuationFreeDoc


#Storing the puntuation free text
dfText['clean_msg']= dfText['original_text'].apply(lambda x:removePunctuation(x))
dfText.head()

,original_text,clean_msg
0,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",MARTINSBURG WVa — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia police said\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County The Journal reported\n\nCody Wade Braithwaite 32 of Winchester Virginia was charged with DUI with death fleeing DUI and being a prohibited person in possession of a firearm and a fugitive from justice Berkeley County Sheriff Curtis Keller said\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail The driver of the car was pronounced dead at the scene The driver’s name was not immediately released\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped Keller said\n\nBraithwaite was being held without bond in the Eastern Regional Jail It wasn’t immediately known whether Braithwaite has an attorney\n\n\n\nInformation from The Journal httpjournalnewsnet\n\nCopyright 2020 The Associated Press All rights reserved This material may not be published broadcast rewritten or redistributed


Transforming to lowercase

In [26]:
dfText['msg_lower']= dfText['clean_msg'].apply(lambda x: x.lower())
dfText.head()

,original_text,clean_msg,msg_lower
0,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",MARTINSBURG WVa — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia police said\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County The Journal reported\n\nCody Wade Braithwaite 32 of Winchester Virginia was charged with DUI with death fleeing DUI and being a prohibited person in possession of a firearm and a fugitive from justice Berkeley County Sheriff Curtis Keller said\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail The driver of the car was pronounced dead at the scene The driver’s name was not immediately released\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped Keller said\n\nBraithwaite was being held without bond in the Eastern Regional Jail It wasn’t immediately known whether Braithwaite has an attorney\n\n\n\nInformation from The Journal httpjournalnewsnet\n\nCopyright 2020 The Associated Press All rights reserved This material may not be published broadcast rewritten or redistributed,martinsburg wva — a suspected drunken driver was arrested in a new year’s day highway crash that killed another motorist in west virginia police said\n\nthe accident occurred early wednesday along southbound interstate 81 in berkeley county the journal reported\n\ncody wade braithwaite 32 of winchester virginia was charged with dui with death fleeing dui and being a prohibited person in possession of a firearm and a fugitive from justice berkeley county sheriff curtis keller said\n\nkeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail the driver of the car was pronounced dead at the scene the driver’s name was not immediately released\n\nthe suspect’s vehicle also hit a deputy’s cruiser before being stopped keller said\n\nbraithwaite was being held without bond in the eastern regional jail it wasn’t immediately known whether braithwaite has an attorney\n\n\n\ninformation from the journal httpjournalnewsnet\n\ncopyright 2020 the associated press all rights reserved this material may not be published broadcast rewritten or redistributed


Tokenization

In [35]:
dfText['msg_tokenized']= dfText['msg_lower'].apply(lambda x: nltk.word_tokenize(x))
dfText.head()

,original_text,clean_msg,msg_lower,msg_tokenied,msg_tokenized
0,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",MARTINSBURG WVa — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia police said\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County The Journal reported\n\nCody Wade Braithwaite 32 of Winchester Virginia was charged with DUI with death fleeing DUI and being a prohibited person in possession of a firearm and a fugitive from justice Berkeley County Sheriff Curtis Keller said\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail The driver of the car was pronounced dead at the scene The driver’s name was not immediately released\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped Keller said\n\nBraithwaite was being held without bond in the Eastern Regional Jail It wasn’t immediately known whether Braithwaite has an attorney\n\n\n\nInformation from The Journal httpjournalnewsnet\n\nCopyright 2020 The Associated Press All rights reserved This material may not be published broadcast rewritten or redistributed,martinsburg wva — a suspected drunken driver was arrested in a new year’s day highway crash that killed another motorist in west virginia police said\n\nthe accident occurred early wednesday along southbound interstate 81 in berkeley county the journal reported\n\ncody wade braithwaite 32 of winchester virginia was charged with dui with death fleeing dui and being a prohibited person in possession of a firearm and a fugitive from justice berkeley county sheriff curtis keller said\n\nkeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail the driver of the car was pronounced dead at the scene the driver’s name was not immediately released\n\nthe suspect’s vehicle also hit a deputy’s cruiser before being stopped keller said\n\nbraithwaite was being held without bond in the eastern regional jail it wasn’t immediately known whether braithwaite has an attorney\n\n\n\ninformation from the journal httpjournalnewsnet\n\ncopyright 2020 the associated press all rights reserved this material may not be published broadcast rewritten or redistributed,"[martinsburg, wva, —, a, suspected, drunken, driver, was, arrested, in, a, new, year, ’, s, day, highway, crash, that, killed, another, motorist, in, west, virginia, police, said, the, accident, occurred, early, wednesday, along, southbound, interstate, 81, in, berkeley, county, the, journal, reported, cody, wade, braithwaite, 32, of, winchester, virginia, was, charged

Stopwords filtering

In [33]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
def removeStopwords(listWords):
    filteredWords = [word for word in listWords if word not in stop_words]
    return filteredWords

dfText['no_stopwords']= dfText['msg_tokenized'].apply(lambda x: removeStopwords(x))
dfText[['msg_tokenized', 'no_stopwords']]


,msg_tokenized,no_stopwords
0,"[martinsburg, wva, —, a, suspected, drunken, driver, was, arrested, in, a, new, year, ’, s, day, highway, crash, that, killed, another, motorist, in, west, virginia, police, said, the, accident, occurred, early, wednesday, along, southbound, interstate, 81, in, berkeley, county, the, journal, reported, cody, wade, braithwaite, 32, of, winchester, virginia, was, charged, with, dui, with, death, fleeing, dui, and, being, a, prohibited, person, in, possession, of, a, firearm, and, a, fugitive, from, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, a, deputy, investigating, calls, about, a, reckless, driver, attempted, to, make, a, traffic, stop, but, the, suspect, ’, s, vehicle, ...]","[martinsburg, wva, —, suspected, drunken, driver, arrested, new, year, ’, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, dui, death, fleeing, dui, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, calls, reckless, driver, attempted, make, traffic, stop, suspect, ’, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, ’, name, immediately, released, suspect, ’, vehicle, also, hit, deputy, ’, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, ...]"


Stemming

In [38]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stemming(listWords):
    stemText = [porter.stem(word) for word in listWords]
    return stemText

dfText['msg_stemmed']= dfText['no_stopwords'].apply(lambda x: stemming(x))
dfText[['no_stopwords', 'msg_stemmed']]

,no_stopwords,msg_stemmed
0,"[martinsburg, wva, —, suspected, drunken, driver, arrested, new, year, ’, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, dui, death, fleeing, dui, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, calls, reckless, driver, attempted, make, traffic, stop, suspect, ’, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, ’, name, immediately, released, suspect, ’, vehicle, also, hit, deputy, ’, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, ...]","[martinsburg, wva, —, suspect, drunken, driver, arrest, new, year, ’, day, highway, crash, kill, anoth, motorist, west, virginia, polic, said, accid, occur, earli, wednesday, along, southbound, interst, 81, berkeley, counti, journal, report, codi, wade, braithwait, 32, winchest, virginia, charg, dui, death, flee, dui, prohibit, person, possess, firearm, fugit, justic, berkeley, counti, sheriff, curti, keller, said, keller, said, deputi, investig, call, reckless, driver, attempt, make, traffic, stop, suspect, ’, vehicl, sped, hit, car, struck, guardrail, driver, car, pronounc, dead, scene, driver, ’, name, immedi, releas, suspect, ’, vehicl, also, hit, deputi, ’, cruiser, stop, keller, said, braithwait, held, without, bond, eastern, ...]"


Lemmatization

In [40]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(listWords):
    lemmText = [wordnet_lemmatizer.lemmatize(word) for word in listWords]
    return lemmText

dfText['msg_lemmatized']= dfText['no_stopwords'].apply(lambda x: lemmatizer(x))
dfText[['no_stopwords', 'msg_stemmed', 'msg_lemmatized']]

,no_stopwords,msg_stemmed,msg_lemmatized
0,"[martinsburg, wva, —, suspected, drunken, driver, arrested, new, year, ’, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, dui, death, fleeing, dui, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, calls, reckless, driver, attempted, make, traffic, stop, suspect, ’, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, ’, name, immediately, released, suspect, ’, vehicle, also, hit, deputy, ’, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, ...]","[martinsburg, wva, —, suspect, drunken, driver, arrest, new, year, ’, day, highway, crash, kill, anoth, motorist, west, virginia, polic, said, accid, occur, earli, wednesday, along, southbound, interst, 81, berkeley, counti, journal, report, codi, wade, braithwait, 32, winchest, virginia, charg, dui, death, flee, dui, prohibit, person, possess, firearm, fugit, justic, berkeley, counti, sheriff, curti, keller, said, keller, said, deputi, investig, call, reckless, driver, attempt, make, traffic, stop, suspect, ’, vehicl, sped, hit, car, struck, guardrail, driver, car, pronounc, dead, scene, driver, ’, name, immedi, releas, suspect, ’, vehicl, also, hit, deputi, ’, cruiser, stop, keller, said, braithwait, held, without, bond, eastern, ...]","[martinsburg, wva, —, suspected, drunken, driver, arrested, new, year, ’, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, duo, death, fleeing, duo, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, call, reckless, driver, attempted, make, traffic, stop, suspect, ’, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, ’, name, immediately, released, suspect, ’, vehicle, also, hit, deputy, ’, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, ...]"
